# How to download flights csv file from transtats website

**In this notebook, we will**
1. Download a csv file for each of your chosen year(s) and month(s)
2. Prepare the data for further processing
3. Push the prepared data to a table in the database




In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files

In [2]:
# Import the get_engine function from our sql_functions.
from sql_functions import get_engine #adjust this as necessary to match your sql_functions.py connection methods

# 1. Download csv file with flight data for your specific year/month

In the following, you are going to download a csv file containing flight data from [this website](https://transtats.bts.gov).    
You can specify, which data you want to download. 

Choose a month/year that you want to explore further.
With the following functions, you will download a csv file on public flight data from [this website](https://transtats.bts.gov) containing data of your chosen month/year.    
The file will be stored in a data folder.
Check out the url from which we download the data(https://transtats.bts.gov/PREZIP). Can we download data in this way from every source? What do you think?

In [3]:
# Specifies path for saving file
path ='data/' 
# Create the data folder
#!mkdir {path}

In [4]:
# function to get specified csv file from the website https://transtats.bts.gov

def download_data(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
    # Download the database
    r = requests.get(f'{url}', verify=False)
    # Save database to local file storage
    with open(path+zip_file, 'wb') as f:
        f.write(r.content)
        print(f'--> zip_file with name: {zip_file} downloaded succesfully.' )

#download_data(2005,8)
    

In [5]:
# function to extract the csv files inside the zip files

def extract_zip(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    with ZipFile(path+zip_file, 'r') as zip_ref:
        zip_ref.extractall(path)
        csv_file =  zip_ref.namelist()[0]
        print(f'--> zip_file was succesfully extracted to: {csv_file}.' )

#extract_zip(2005,8)

Don't worry - the following download of the data you chose may take some time ...

In [6]:
# years_list = [2020, 2021] # list of years you want to look at (can of course also be a single year)
# months_list = [1, 2] # list of months you want to look at (can of course also be a single month)

# download flights data as zipfile(s)
# we use a nested loop to specify the years and months to define the range of the data we would like to have 
#for year in years_list:
#    for month in months_list:
#        download_data(year, month)
#        extract_zip(year, month)

Now it is time to load the csv files into dataframes. You can create your own function equivalent to the functions above. But you need to decide whether...

- Do you need one dataframe for every month?
- Would you like to proceed with only one dataframe containing all the data you downloaded?
- One dataframe for every year?

There are certain things to consider before.
- changing column names
- dealing with missing data
- changing datatypes

In [7]:
# define the name of the csv file you want to read in
csv_file = 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_8.csv'

# Read in your data
df = pd.read_csv(path+csv_file, low_memory = False)
display(df.shape)
display(df.head())

(630904, 110)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2005,3,8,20,6,2005-08-20,DL,19790,DL,N908DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,3,8,20,6,2005-08-20,DL,19790,DL,N112DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,3,8,20,6,2005-08-20,DL,19790,DL,N989DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,3,8,20,6,2005-08-20,DL,19790,DL,N989DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,3,8,20,6,2005-08-20,DL,19790,DL,N947DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'DOT_ID_Reporting_Airline',
       'IATA_CODE_Reporting_Airline', 'Tail_Number',
       ...
       'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID',
       'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff',
       'Div5TailNum', 'Unnamed: 109'],
      dtype='object', length=110)

In [9]:
df.info

<bound method DataFrame.info of         Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0       2005        3      8          20          6  2005-08-20   
1       2005        3      8          20          6  2005-08-20   
2       2005        3      8          20          6  2005-08-20   
3       2005        3      8          20          6  2005-08-20   
4       2005        3      8          20          6  2005-08-20   
...      ...      ...    ...         ...        ...         ...   
630899  2005        3      8          19          5  2005-08-19   
630900  2005        3      8          20          6  2005-08-20   
630901  2005        3      8          21          7  2005-08-21   
630902  2005        3      8          22          1  2005-08-22   
630903  2005        3      8          23          2  2005-08-23   

       Reporting_Airline  DOT_ID_Reporting_Airline  \
0                     DL                     19790   
1                     DL                     19790   
2

# 2. Prepare the csv file for further processing

In the next step, we clean and prepare our dataset.

a) Since the dataset consists of a lot of columns, we we define which ones to keep.  
(Same as known from our SQL exercises on flights data.)

In [10]:
# Columns from downloaded file that are to be kept

columns_to_keep = [
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'ActualElapsedTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [11]:
# The columns in the database have different naming as in the source csv files. Lets get the names from the database

schema = 'cgn_analytics_22_3' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database

table_name_sql = f'''SELECT COLUMN_NAME 
                    FROM INFORMATION_SCHEMA.COLUMNS 
                    WHERE TABLE_NAME = 'flights'
                    AND TABLE_SCHEMA ='{schema}'
                    ORDER BY ordinal_position'''
c_names = engine.execute(table_name_sql).fetchall()
c_names

[('flight_date',),
 ('dep_time',),
 ('sched_dep_time',),
 ('dep_delay',),
 ('arr_time',),
 ('sched_arr_time',),
 ('arr_delay',),
 ('airline',),
 ('tail_number',),
 ('flight_number',),
 ('origin',),
 ('dest',),
 ('air_time',),
 ('actual_elapsed_time',),
 ('distance',),
 ('cancelled',),
 ('diverted',)]

In [12]:
# we can clean up the results into a clean list
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names        

['flight_date',
 'dep_time',
 'sched_dep_time',
 'dep_delay',
 'arr_time',
 'sched_arr_time',
 'arr_delay',
 'airline',
 'tail_number',
 'flight_number',
 'origin',
 'dest',
 'air_time',
 'actual_elapsed_time',
 'distance',
 'cancelled',
 'diverted']

In [13]:
# Just in case the above fails here are the results
'''new_column_names_alternate = ['flight_date', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 
                'arr_delay', 'airline', 'tail_number', 'flight_number', 'origin', 'dest', 'air_time', 'actual_elapsed_time', 'distance', 'cancelled', 'diverted' ]'''

"new_column_names_alternate = ['flight_date', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', \n                'arr_delay', 'airline', 'tail_number', 'flight_number', 'origin', 'dest', 'air_time', 'actual_elapsed_time', 'distance', 'cancelled', 'diverted' ]"

b) With the next function, we make our csv file ready to be uploaded to SQL.  
We only keep to above specified columns and convert the datatypes.

In [14]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    df_airline['ActualElapsedTime']= pd.to_numeric(df_airline['ActualElapsedTime'], downcast='integer', errors='coerce')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [15]:
# Call function and check resulting dataframe
df_clean = clean_airline_df(df)
df_clean.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
0,2005-08-20,728.0,720,8.0,1044.0,1016,28.0,DL,N908DL,992,SEA,SLC,92.0,136.0,689.0,0,0
1,2005-08-20,2036.0,2039,-3.0,2231.0,2239,-8.0,DL,N112DL,994,MIA,ATL,81.0,115.0,595.0,0,0
2,2005-08-20,749.0,750,-1.0,851.0,905,-14.0,DL,N989DL,995,CHS,ATL,54.0,62.0,259.0,0,0
3,2005-08-20,1024.0,1025,-1.0,1244.0,1248,-4.0,DL,N989DL,995,ATL,BDL,118.0,140.0,859.0,0,0
4,2005-08-20,732.0,735,-3.0,933.0,946,-13.0,DL,N947DL,997,MCO,CVG,105.0,121.0,756.0,0,0


In [16]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630904 entries, 0 to 630903
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   flight_date          630904 non-null  datetime64[ns]
 1   dep_time             617452 non-null  float64       
 2   sched_dep_time       630904 non-null  int64         
 3   dep_delay            617452 non-null  float64       
 4   arr_time             615643 non-null  float64       
 5   sched_arr_time       630904 non-null  int16         
 6   arr_delay            615643 non-null  float64       
 7   airline              630904 non-null  object        
 8   tail_number          623962 non-null  object        
 9   flight_number        630904 non-null  int64         
 10  origin               630904 non-null  object        
 11  dest                 630904 non-null  object        
 12  air_time             615643 non-null  float64       
 13  actual_elapsed

If you decide to only look at specific airports, it is a good decision to filter for them in advance.  
This function does the filtering. 

In [17]:
# Specify the airports you are interested in and put them as a list in the function.
def select_airport(df, airports):
    ''' Helper function for filtering the airline dataframe for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out

In [18]:
# Execute function, filtering for New York area airports
airports=['MSY', 'BTR', 'GPT', 'MOB', 'PNS', 'VPS']
if len(airports) > 0:
    df_selected_airports = select_airport(df_clean, airports)
else:
    df_selected_airports = df_clean
    
df_selected_airports.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
20,2005-08-20,1217.0,1125,52.0,1440.0,1349,51.0,DL,N998DL,1013,MSY,ATL,67.0,83.0,425.0,0,0
30,2005-08-20,1928.0,1930,-2.0,2147.0,2159,-12.0,DL,N922DL,1023,MSY,ATL,62.0,79.0,425.0,0,0
31,2005-08-20,723.0,725,-2.0,947.0,1006,-19.0,DL,N909DA,1024,MSY,SLC,182.0,204.0,1428.0,0,0
110,2005-08-20,1214.0,1215,-1.0,1223.0,1232,-9.0,DL,N319DL,1090,ATL,GPT,50.0,69.0,352.0,0,0
120,2005-08-20,1120.0,1116,4.0,1125.0,1123,2.0,DL,N984DL,1100,ATL,VPS,41.0,65.0,250.0,0,0


In [19]:
df_selected_airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16331 entries, 20 to 628311
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   flight_date          16331 non-null  datetime64[ns]
 1   dep_time             14565 non-null  float64       
 2   sched_dep_time       16331 non-null  int64         
 3   dep_delay            14565 non-null  float64       
 4   arr_time             14444 non-null  float64       
 5   sched_arr_time       16331 non-null  int16         
 6   arr_delay            14444 non-null  float64       
 7   airline              16331 non-null  object        
 8   tail_number          15286 non-null  object        
 9   flight_number        16331 non-null  int64         
 10  origin               16331 non-null  object        
 11  dest                 16331 non-null  object        
 12  air_time             14444 non-null  float64       
 13  actual_elapsed_time  14444 no

In [20]:
df_selected_airports.isnull().sum()

flight_date               0
dep_time               1766
sched_dep_time            0
dep_delay              1766
arr_time               1887
sched_arr_time            0
arr_delay              1887
airline                   0
tail_number            1045
flight_number             0
origin                    0
dest                      0
air_time               1887
actual_elapsed_time    1887
distance                  0
cancelled                 0
diverted                  0
dtype: int64

In [21]:
df_selected_airports.describe()

,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,flight_number,air_time,actual_elapsed_time,distance,cancelled,diverted
count,14565.000000,16331.000000,14565.000000,14444.000000,16331.000000,14444.000000,16331.000000,14444.000000,14444.000000,16331.000000,16331.000000,16331.000000
mean,1332.226021,1326.938767,11.261105,1478.710953,1493.337334,10.763223,1943.070296,81.872127,102.725768,566.346886,0.108138,0.007409
std,472.301522,453.502750,33.914630,485.597797,452.988233,35.499128,1463.851786,39.902077,41.784756,335.098985,0.310564,0.085760
min,3.000000,500.000000,-1199.000000,1.000000,425.000000,-42.000000,4.000000,15.000000,43.000000,250.000000,0.000000,0.000000
25%,931.000000,932.000000,-3.000000,1059.000000,1109.000000,-7.000000,716.000000,53.000000,73.000000,321.000000,0.000000,0.000000
50%,1311.000000,1314.000000,0.000000,1452.000000,1500.000000,0.000000,1568.000000,67.000000,89.000000,448.000000,0.000000,0.000000
75%,1730.000000,1716.000000,11.000000,1869.250000,1850.000000,15.000000,2929.000000,94.000000,118.000000,651.000000,0.000000,0.000000
max,2400.000000,2323.000000,799.000000,2400.000000,2355.000000,806.000000,5600.000000,256.000000,374.000000,1911.000000,1.000000,1.000000


In [22]:
# find out if cancelled matches dep_delay=NULL
# f_planes['tail_number'].str[1:].isin(all['nnum']).value_counts()

# 3. Push the prepared data to a table in the database

In [23]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'flights_katrina'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         df_selected_airports.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schmea that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [24]:
# Just to be sure: Check if the number of rows match
table_name_sql = f'''SELECT count(*) 
                    FROM {schema}.{table_name}
                    '''
engine.execute(table_name_sql).fetchall()[0][0] == df_selected_airports.shape[0]

True

In [25]:
from sql_functions import get_dataframe

In [26]:
d_airports = get_dataframe('SELECT * FROM cgn_analytics_22_3.airports')
d_airports.head()

,faa,name,lat,lon,alt,tz,dst,city,country
0,GKA,Goroka Airport,-6.081690,145.391998,5282,10.0,U,Goroka,Papua New Guinea
1,MAG,Madang Airport,-5.207080,145.789001,20,10.0,U,Madang,Papua New Guinea
2,HGU,Mount Hagen Kagamuga Airport,-5.826790,144.296005,5388,10.0,U,Mount Hagen,Papua New Guinea
3,LAE,Nadzab Airport,-6.569803,146.725977,239,10.0,U,Nadzab,Papua New Guinea
4,POM,Port Moresby Jacksons International Airport,-9.443380,147.220001,146,10.0,U,Port Moresby,Papua New Guinea


In [27]:
d_airports.drop(columns=['alt', 'tz', 'dst', 'country'], inplace=True)
d_airports.rename(columns={'faa': 'origin'}, inplace=True)
d_airports.columns

Index(['origin', 'name', 'lat', 'lon', 'city'], dtype='object')

In [28]:
flights_katrina_airports = df_selected_airports.merge(d_airports, how='left', on='origin')
flights_katrina_airports.info()
flights_katrina_airports.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16331 entries, 0 to 16330
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   flight_date          16331 non-null  datetime64[ns]
 1   dep_time             14565 non-null  float64       
 2   sched_dep_time       16331 non-null  int64         
 3   dep_delay            14565 non-null  float64       
 4   arr_time             14444 non-null  float64       
 5   sched_arr_time       16331 non-null  int16         
 6   arr_delay            14444 non-null  float64       
 7   airline              16331 non-null  object        
 8   tail_number          15286 non-null  object        
 9   flight_number        16331 non-null  int64         
 10  origin               16331 non-null  object        
 11  dest                 16331 non-null  object        
 12  air_time             14444 non-null  float64       
 13  actual_elapsed_time  14444 non-

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,...,dest,air_time,actual_elapsed_time,distance,cancelled,diverted,name,lat,lon,city
0,2005-08-20,1217.0,1125,52.0,1440.0,1349,51.0,DL,N998DL,1013,...,ATL,67.0,83.0,425.0,0,0,Louis Armstrong New Orleans International Airport,29.993401,-90.258003,New Orleans
1,2005-08-20,1928.0,1930,-2.0,2147.0,2159,-12.0,DL,N922DL,1023,...,ATL,62.0,79.0,425.0,0,0,Louis Armstrong New Orleans International Airport,29.993401,-90.258003,New Orleans
2,2005-08-20,723.0,725,-2.0,947.0,1006,-19.0,DL,N909DA,1024,...,SLC,182.0,204.0,1428.0,0,0,Louis Armstrong New Orleans International Airport,29.993401,-90.258003,New Orleans
3,2005-08-20,1214.0,1215,-1.0,1223.0,1232,-9.0,DL,N319DL,1090,...,GPT,50.0,69.0,352.0,0,0,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,Atlanta
4,2005-08-20,1120.0,1116,4.0,1125.0,1123,2.0,DL,N984DL,1100,...,VPS,41.0,65.0,250.0,0,0,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,Atlanta
